In [52]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
import os

import requests
import zipfile
import io
import os

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq

In [53]:
# List of country codes
cntry_names = ['AUS', 'AUT', 'BEL', 'BRA', 'CAN', 'CHN', 'COL', 'DNK', 'FIN', 'FRA', 'DEU',
               'GRC', 'IND', 'IDN', 'IRL', 'ITA', 'JPN', 'KOR', 'MEX', 'NOR', 'POL', 'PRT',
               'RUS', 'ESP', 'SWE', 'CHE', 'THA', 'TUR', 'GBR', 'USA']

# Mapping of country names to numeric values
cntry_name_to_code = {
    'AUS': 1, 'AUT': 2, 'BEL': 3, 'BRA': 4, 'CAN': 5, 'CHN': 6, 'COL': 7, 'DNK': 8, 'FIN': 9, 'FRA': 10,
    'DEU': 11, 'GRC': 12, 'IND': 13, 'IDN': 14, 'IRL': 15, 'ITA': 16, 'JPN': 17, 'KOR': 18, 'MEX': 19,
    'NOR': 20, 'POL': 21, 'PRT': 22, 'RUS': 23, 'ESP': 24, 'SWE': 25, 'CHE': 26, 'THA': 27, 'TUR': 28,
    'GBR': 29, 'USA': 30
}

In [54]:
# Directory to save the extracted files
input_dir = "./gravity-data/"

# Path to the Stata file
stata_file_path = input_dir + "dist_cepii.dta"

# Load the Stata file into a DataFrame
df = pd.read_stata(stata_file_path)

# Filter the DataFrame to only include rows where 'iso_o' or 'iso_d' is in the cntry_names list
df = df[df['iso_o'].isin(cntry_names) & df['iso_d'].isin(cntry_names)]

# Display the first few rows of the DataFrame
df.head(10)

df.dist = 1 /1.609*df.dist # convert km to miles


In [55]:
# Mapping of country names to numeric values
cntry_name_to_code = {
    'AUS': 1, 'AUT': 2, 'BEL': 3, 'BRA': 4, 'CAN': 5, 'CHN': 6, 'COL': 7, 'DNK': 8, 'FIN': 9, 'FRA': 10,
    'DEU': 11, 'GRC': 12, 'IND': 13, 'IDN': 14, 'IRL': 15, 'ITA': 16, 'JPN': 17, 'KOR': 18, 'MEX': 19,
    'NOR': 20, 'POL': 21, 'PRT': 22, 'RUS': 23, 'ESP': 24, 'SWE': 25, 'CHE': 26, 'THA': 27, 'TUR': 28,
    'GBR': 29, 'USA': 30
}

# Map iso_o to neccode
df['neccode'] = df['iso_o'].map(cntry_name_to_code)

# Replace missing neccode with 777
df['neccode'].fillna(777, inplace=True)

# Map iso_o to neccode
df['niccode'] = df['iso_d'].map(cntry_name_to_code)

# Replace missing neccode with 777
df['niccode'].fillna(777, inplace=True)

# Drop rows where neccode is 777
df = df[df['neccode'] != 777]

# Drop rows where neccode is 777
df = df[df['niccode'] != 777]

# Display the first few rows to verify
df.head()

,iso_o,iso_d,contig,comlang_off,comlang_ethno,colony,comcol,curcol,col45,smctry,dist,distcap,distw,distwces,neccode,niccode
2475,AUS,AUS,0,0,0,0,0,0,0,0,648.114807,1042.816650,1121.103649,153.628704,1,1
2476,AUS,AUT,0,0,0,0,0,0,0,0,9937.199219,15931.753906,15608.418243,15559.910589,1,2
2479,AUS,BEL,0,0,0,0,0,0,0,0,10416.159180,16734.728516,16319.186512,16277.454792,1,3
2491,AUS,BRA,0,0,0,0,0,0,0,0,8310.601562,14070.799805,14046.748413,13983.355490,1,4
2497,AUS,CAN,0,1,1,0,0,0,0,0,9687.171875,16123.001953,15391.073574,15223.987629,1,5


In [56]:
# Define the conditions and corresponding values for the distance bins
conditions = [
    (df['dist'] <= 375),
    (df['dist'] > 375) & (df['dist'] <= 750),
    (df['dist'] > 750) & (df['dist'] <= 1500),
    (df['dist'] > 1500) & (df['dist'] <= 3000),
    (df['dist'] > 3000) & (df['dist'] <= 6000),
    (df['dist'] > 6000)
]

# Define the values for each condition
values = [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]

# Create the distbin column using numpy.select
df['distbin'] = np.select(conditions, values, default=0.0)

# Create binary columns for each distance bin
df['bin375'] = (df['dist'] <= 375).astype(int)
df['bin750'] = ((df['dist'] > 375) & (df['dist'] <= 750)).astype(int)
df['bin1500'] = ((df['dist'] > 750) & (df['dist'] <= 1500)).astype(int)
df['bin3000'] = ((df['dist'] > 1500) & (df['dist'] <= 3000)).astype(int)
df['bin6000'] = ((df['dist'] > 3000) & (df['dist'] <= 6000)).astype(int)
df['binmax'] = (df['dist'] > 6000).astype(int)

df.rename(columns={"niccode": 'importer', 'neccode': 'exporter', "contig": "border"}, inplace=True)

In [57]:
df.head()

,iso_o,iso_d,border,comlang_off,comlang_ethno,colony,comcol,curcol,col45,smctry,...,distwces,exporter,importer,distbin,bin375,bin750,bin1500,bin3000,bin6000,binmax
2475,AUS,AUS,0,0,0,0,0,0,0,0,...,153.628704,1,1,2.0,0,1,0,0,0,0
2476,AUS,AUT,0,0,0,0,0,0,0,0,...,15559.910589,1,2,6.0,0,0,0,0,0,1
2479,AUS,BEL,0,0,0,0,0,0,0,0,...,16277.454792,1,3,6.0,0,0,0,0,0,1
2491,AUS,BRA,0,0,0,0,0,0,0,0,...,13983.355490,1,4,6.0,0,0,0,0,0,1
2497,AUS,CAN,0,1,1,0,0,0,0,0,...,15223.987629,1,5,6.0,0,0,0,0,0,1


In [58]:
df = df.sort_values(by=["exporter", "importer"])

df = df[["iso_o", "iso_d","border", "dist", "importer", "exporter", 
         "distbin", "bin375", "bin750", "bin1500", "bin3000", "bin6000", "binmax"]]

In [59]:
df.tail(30)

,iso_o,iso_d,border,dist,importer,exporter,distbin,bin375,bin750,bin1500,bin3000,bin6000,binmax
46827,USA,AUS,0,9949.530273,1,30,6.0,0,0,0,0,0,1
46828,USA,AUT,0,4225.419922,2,30,5.0,0,0,0,0,1,0
46831,USA,BEL,0,3661.722656,3,30,5.0,0,0,0,0,1,0
46843,USA,BRA,0,4782.042969,4,30,5.0,0,0,0,0,1,0
46849,USA,CAN,1,340.829468,5,30,1.0,1,0,0,0,0,0
46853,USA,CHN,0,6832.619629,6,30,6.0,0,0,0,0,0,1
46858,USA,COL,0,2499.174316,7,30,4.0,0,0,0,1,0,0
46870,USA,DNK,0,3848.286377,8,30,5.0,0,0,0,0,1,0
46880,USA,FIN,0,4118.038086,9,30,5.0,0,0,0,0,1,0
46883,USA,FRA,0,3628.438477,10,30,5.0,0,0,0,0,1,0


In [60]:
df.to_parquet("top30_gravity_data.parquet", index=False)

df.to_csv("top30_gravity_data.csv", index=False)

In [61]:
df.head()

,iso_o,iso_d,border,dist,importer,exporter,distbin,bin375,bin750,bin1500,bin3000,bin6000,binmax
2475,AUS,AUS,0,648.114807,1,1,2.0,0,1,0,0,0,0
2476,AUS,AUT,0,9937.199219,2,1,6.0,0,0,0,0,0,1
2479,AUS,BEL,0,10416.159180,3,1,6.0,0,0,0,0,0,1
2491,AUS,BRA,0,8310.601562,4,1,6.0,0,0,0,0,0,1
2497,AUS,CAN,0,9687.171875,5,1,6.0,0,0,0,0,0,1
